## References and important URLS requirements- 

- [1] https://openai.com/blog/function-calling-and-other-api-updates
- [2] https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb
- [3] https://rapidapi.com/weatherapi/api/weatherapi-com/
- [4] https://platform.openai.com/account/api-keys

In [3]:
import os
ROOT = "/Users/Admin/Desktop/playground/YouTube/demo/gpt-function-calling-demo/"
os.chdir(ROOT)
os.getcwd()

'c:\\Users\\Admin\\Desktop\\playground\\YouTube\\demo\\gpt-function-calling-demo'

# Previous ways - 

## Via Openai Python ChatCompletion class- 

```python
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)
```

## Via CURL command

```bash
curl https://api.openai.com/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-3.5-turbo",
    "messages": [
      {"role": "system", "content": "You are a helpful assistant."}, 
      {"role": "user", "content": "Hello!"}
      ]
  }'
```

## Via requests module

```python
import requests

headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + OPENAI_API_KEY,
}
json_data = {
    "model": "gpt-3.5-turbo", 
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."}, 
        {"role": "user", "content": "Hello!"}
        ]}
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=json_data,
)
```


## Weather Chatbot example

In [4]:
import requests
from dotenv import load_dotenv
import json
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GPT_MODEL = "gpt-3.5-turbo-0613" # new model for function calling abilities

## Checkout the following example - 

```bash
curl https://api.openai.com/v1/chat/completions -u :$OPENAI_API_KEY -H 'Content-Type: application/json' -d '{
  "model": "gpt-3.5-turbo-0613",
  "messages": [
    {"role": "user", "content": "What is the weather like in Boston?"}
  ],
  "functions": [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]
}'
```

focus on the function keyword and the details under it can be found in a functions docstring. So from the above function description,
you can assume the `get_current_weather` as follows - 

```python
def get_current_weather(location, unit):
    """Get the current weather in a given location

    Args:
        location (str): The city and state, e.g. San Francisco, CA
        unit (str): ["celsius", "fahrenheit"]
    """
    pass
```

In [5]:
import requests

def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + OPENAI_API_KEY,
    }
    json_data = {
        "model": model, 
        "messages": messages
        }
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )

        return response
    except Exception as e:
        print(f"\n>>>> Unable to generate ChatCompletion response")
        print(f"\n>>>> Exception: {e}")
        raise e

In [6]:
# simple use

sample_messages = [
    {"role": "system", "content": "you are a helpful assistant"},
    {"role": "user", "content": "Hi there"},
]

chat_response = chat_completion_request(messages=sample_messages)

chat_response

<Response [200]>

In [7]:
chat_response.json()

{'id': 'chatcmpl-7SBheX8YV9n6tPA2unwJbA2VcqlVa',
 'object': 'chat.completion',
 'created': 1686952526,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hello! How can I assist you today?'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 18, 'completion_tokens': 9, 'total_tokens': 27}}

In [8]:
assistant_message = chat_response.json()["choices"][0]["message"]
assistant_message

{'role': 'assistant', 'content': 'Hello! How can I assist you today?'}

In [9]:
creator_name = "c17hawke"

system_instruction = f"""
You are Weather bot created by {creator_name}, \
You first greet the customer, and you MUST introduce yourself as -
"Hi I am Weather bot created by {creator_name}! How may I assist you today?" \
an automated service to tell Weather infomation \
based on the location provided by the user. \
then collects the location information\
Don't make assumptions about what \
values to plug into functions. \
Ask for clarification if a user request is ambiguous.
You respond in a short, very conversational friendly style. \
"""

In [10]:
messages = [
    {"role": "system", "content": system_instruction}
]

In [11]:
functions = [
    {
        "name": "get_current_weather", # name of your function
        "description": "Get the current weather", # description of your function
        "parameters": {
            "type": "object",
            "properties": { # 1st argument of your function
                "location": { # argument name
                    "type": "string", # argument type
                    "description": "The city and state, e.g. San Francisco, CA", # description of your function with example
                }
            },
            "required": ["location"], # required argument of your
        },
    }
]

In [12]:
def get_current_weather(location):
    RAPID_API_KEY = os.getenv("RAPID_API_KEY")
    try:
        url = "https://weatherapi-com.p.rapidapi.com/current.json"

        querystring = {"q": location}
        print(f"\n>>>> got the querystring as: {querystring}")
        headers = {
            "X-RapidAPI-Key": RAPID_API_KEY,
            "X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)
        response_json = response.json()
        print(f"\n>>>> got the RAPID API response as: \n{response_json}")
        return response_json
    except Exception as e:
        raise e

In [13]:
user_message = "Hi there!"
messages.append({"role": "user", "content": user_message})
chat_response = chat_completion_request(messages = messages, functions = functions)

In [14]:
chat_response

<Response [200]>

In [15]:
chat_response.json()

{'id': 'chatcmpl-7SBnnzcZfy4hV45EGpcg4saM4HQVg',
 'object': 'chat.completion',
 'created': 1686952907,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hi there! How may I assist you today?'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 167, 'completion_tokens': 11, 'total_tokens': 178}}

In [16]:
assistant_message = chat_response.json()["choices"][0]["message"]
assistant_message

{'role': 'assistant', 'content': 'Hi there! How may I assist you today?'}

# Start fresh

In [17]:
messages = list()
messages.append({"role": "system", "content": system_instruction})


user_message = "Hi there!"
messages.append({"role": "user", "content": user_message})
chat_response = chat_completion_request(messages = messages, functions = functions)
print(f"\n>>>> complete_chat_response: \n{chat_response.json()}")
assistant_message = chat_response.json()["choices"][0]["message"]
print(f"\n>>>> assistant message: \n{assistant_message}")


>>>> complete_chat_response: 
{'id': 'chatcmpl-7SBpR21AHJbR6eYlIurjlGCrlzMIP', 'object': 'chat.completion', 'created': 1686953009, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Hi! How can I assist you today? I am Weather bot created by c17hawke!'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 167, 'completion_tokens': 22, 'total_tokens': 189}}

>>>> assistant message: 
{'role': 'assistant', 'content': 'Hi! How can I assist you today? I am Weather bot created by c17hawke!'}


In [18]:
messages.append(assistant_message) # to give context
messages

[{'role': 'system',
  'content': '\nYou are Weather bot created by c17hawke, You first greet the customer, and you MUST introduce yourself as -\n"Hi I am Weather bot created by c17hawke! How may I assist you today?" an automated service to tell Weather infomation based on the location provided by the user. then collects the location informationDon\'t make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\nYou respond in a short, very conversational friendly style. '},
 {'role': 'user', 'content': 'Hi there!'},
 {'role': 'assistant',
  'content': 'Hi! How can I assist you today? I am Weather bot created by c17hawke!'}]

In [19]:
user_message = "I want to know the weather at Bengaluru"
messages.append({"role": "user", "content": user_message})
chat_response = chat_completion_request(messages = messages, functions = functions)
print(f"\n>>>> complete_chat_response: \n{chat_response.json()}\n")
assistant_message = chat_response.json()["choices"][0]["message"]
print(f"\n>>>> assistant message: \n{assistant_message}")


>>>> complete_chat_response: 
{'id': 'chatcmpl-7SBqkYQrKEfqNePyqVVOnH494HfvP', 'object': 'chat.completion', 'created': 1686953090, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "Bengaluru"\n}'}}, 'finish_reason': 'function_call'}], 'usage': {'prompt_tokens': 205, 'completion_tokens': 19, 'total_tokens': 224}}


>>>> assistant message: 
{'role': 'assistant', 'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "Bengaluru"\n}'}}


In [20]:
assistant_message.get("function_call")

{'name': 'get_current_weather', 'arguments': '{\n  "location": "Bengaluru"\n}'}

In [21]:
assistant_message.get("function_call").get("name") # string


'get_current_weather'

In [22]:
assistant_message.get("function_call").get("arguments") # string


'{\n  "location": "Bengaluru"\n}'

In [23]:
import json

def execute_function_call(assistant_message):
    if assistant_message.get("function_call").get("name") == "get_current_weather":
        location = json.loads(assistant_message.get("function_call").get("arguments") )["location"]
        results = get_current_weather(location)
    else:
        results = f"Error: function {assistant_message['function_call']['name']} does not exist"

    return results

In [24]:
results = execute_function_call(assistant_message)
content = json.dumps(results)
content


>>>> got the querystring as: {'q': 'Bengaluru'}

>>>> got the RAPID API response as: 
{'location': {'name': 'Bengaluru', 'region': 'Karnataka', 'country': 'India', 'lat': 12.98, 'lon': 77.58, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1686953483, 'localtime': '2023-06-17 3:41'}, 'current': {'last_updated_epoch': 1686952800, 'last_updated': '2023-06-17 03:30', 'temp_c': 23.0, 'temp_f': 73.4, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 9.4, 'wind_kph': 15.1, 'wind_degree': 260, 'wind_dir': 'W', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 83, 'cloud': 25, 'feelslike_c': 25.0, 'feelslike_f': 77.0, 'vis_km': 6.0, 'vis_miles': 3.0, 'uv': 1.0, 'gust_mph': 14.1, 'gust_kph': 22.7}}


'{"location": {"name": "Bengaluru", "region": "Karnataka", "country": "India", "lat": 12.98, "lon": 77.58, "tz_id": "Asia/Kolkata", "localtime_epoch": 1686953483, "localtime": "2023-06-17 3:41"}, "current": {"last_updated_epoch": 1686952800, "last_updated": "2023-06-17 03:30", "temp_c": 23.0, "temp_f": 73.4, "is_day": 0, "condition": {"text": "Partly cloudy", "icon": "//cdn.weatherapi.com/weather/64x64/night/116.png", "code": 1003}, "wind_mph": 9.4, "wind_kph": 15.1, "wind_degree": 260, "wind_dir": "W", "pressure_mb": 1012.0, "pressure_in": 29.88, "precip_mm": 0.0, "precip_in": 0.0, "humidity": 83, "cloud": 25, "feelslike_c": 25.0, "feelslike_f": 77.0, "vis_km": 6.0, "vis_miles": 3.0, "uv": 1.0, "gust_mph": 14.1, "gust_kph": 22.7}}'

In [25]:
def get_natural_response(content):
    convert_prompt = f"convert this results from weather api to a natural english sentence: {content}"
    messages.append({"role": "user", "content": convert_prompt})
    convert_prompt_response = chat_completion_request(messages=messages)
    print(f"\n>>>> recieved message: {convert_prompt_response.json()}")
    new_assistant_message = convert_prompt_response.json()["choices"][0]["message"]
    messages.append(new_assistant_message)
    content = new_assistant_message["content"]
    print(f"\n>>>> natural response: \n{content}")
    return content


content = get_natural_response(content)



>>>> recieved message: {'id': 'chatcmpl-7SC1MGecGCbMAWscFBsKYVHQ8ChbK', 'object': 'chat.completion', 'created': 1686953748, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'In Bengaluru, Karnataka, India, it is currently partly cloudy. The temperature is 23.0 degrees Celsius (73.4 degrees Fahrenheit) and it feels like 25.0 degrees Celsius (77.0 degrees Fahrenheit). The wind is blowing from the west at a speed of 15.1 kilometers per hour (9.4 miles per hour). The pressure is 1012.0 millibars and the humidity is 83%. The visibility is 6.0 kilometers (3.0 miles) and the UV index is 1.0.'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 499, 'completion_tokens': 115, 'total_tokens': 614}}

>>>> natural response: 
In Bengaluru, Karnataka, India, it is currently partly cloudy. The temperature is 23.0 degrees Celsius (73.4 degrees Fahrenheit) and it feels like 25.0 degrees Celsius (77.0 degrees Fahrenheit). The wind is blowing from t

In [26]:
messages

[{'role': 'system',
  'content': '\nYou are Weather bot created by c17hawke, You first greet the customer, and you MUST introduce yourself as -\n"Hi I am Weather bot created by c17hawke! How may I assist you today?" an automated service to tell Weather infomation based on the location provided by the user. then collects the location informationDon\'t make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\nYou respond in a short, very conversational friendly style. '},
 {'role': 'user', 'content': 'Hi there!'},
 {'role': 'assistant',
  'content': 'Hi! How can I assist you today? I am Weather bot created by c17hawke!'},
 {'role': 'user', 'content': 'I want to know the weather at Bengaluru'},
 {'role': 'user',
  'content': 'convert this results from weather api to a natural english sentence: {"location": {"name": "Bengaluru", "region": "Karnataka", "country": "India", "lat": 12.98, "lon": 77.58, "tz_id": "Asia/Kolkata", "localtime_